In [5]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

os.chdir('C:\\D\\DSPractice\\simple_app')

def log_production_model(config_path):
    config = read_params(config_path)
    
    mlflow_config=config["mlflow_config"]
    
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_registry_uri(remote_server_uri)
    run = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id= runs[runs["metrics.mae"]==lowest]["run_id"][0]
    
    client = MlflowClient()
    for mv in client.search_model_version(f"name='{model_name}'"):
        mv = dict(mv)
        pprint(mv)

log_production_model("params.yaml")


TypeError: join() argument must be str or bytes, not 'int'

In [2]:
pwd

'C:\\D\\DSPractice\\simple_app\\notebooks'